# Deberta fine-tunning in mnli ( original + negative sentences ) 

## preparation

In [ ]:
!nvidia-smi

Wed Aug 31 15:36:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/memoire'

/content/drive/MyDrive/memoire


In [ ]:
%%bash
pip install datasets
pip install transformers
pip install deberta
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=3e76fe64259a6f7a03a8da5762dc1399a7ff9adcf1be053f119363cb6e0b0bb0
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=fe184f4d2ebf832de2c5974344c29d288eab393a50482b159363cd9e7952ab8f
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built seqeva

In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AdamW
import evaluate
from torch.utils.data import DataLoader

## evaluate debera_mnli

In fact, there is one ready to use

### prepare the model and the tokenizer

In [ ]:
# download the checkpoint
checkpoint = "microsoft/deberta-base-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### test

In [ ]:
raw_inputs = [
    "I love this.",
    "I like this so much!",
]
inputs = tokenizer(raw_inputs[0], raw_inputs[1], padding=True, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[  1, 100, 657,  42,   4,   2, 100, 101,  42,  98, 203, 328,   2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model(**inputs.to(model.device))
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-4.6841,  0.9152,  3.5281]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
print(outputs.logits.shape)

torch.Size([1, 3])


In [ ]:
label_map = model.config.id2label
label_map

{0: 'CONTRADICTION', 1: 'NEUTRAL', 2: 'ENTAILMENT'}

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
print(label_map[predictions])

ENTAILMENT


### evaluate on dev

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('corpus/MNLI/dev_mismatched.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# Accuracy:  0.9131407648494712

| Accuracy:  0.8855777054515866


In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('data/mnli_completed_with_importance_neg_only.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[2], tokens[3], tokens[4]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# Accuracy:  0.7745

| Accuracy:  0.6826666666666666


## Fine-tunning in a negative sentences dataset

### Download the datasets

In [ ]:
data_files = './data/mnli_completed_with_importance_neg_only.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ca54e46e4a32349c/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 1500
})

In [ ]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'index': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None)}

In [ ]:
label_map = {'contradiction':0 , 'neutral':1 , 'entailment':2}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [ ]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [ ]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 1500
})

In [ ]:
neg_datasets[1]

{'Unnamed: 0': 1,
 'index': 2,
 'text': 'The second thing I learned is the importance of collaboration among judges, bar associations, private attorneys and civil legal service providers.',
 'hypothesis': 'Everyone who works in law must not work together.',
 'importance': '1',
 'labels': 0}

In [ ]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [ ]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [ ]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 105
    })
})

### prepare trainer evironment

In [ ]:
# download the checkpoint
# checkpoint = "microsoft/deberta-base-mnli"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "deberta-base-fine-tunned-neg-mnli", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mnli")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: importance, index, hypothesis, text, Unnamed: 0. If importance, index, hypothesis, text, Unnamed: 0 are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 945
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 90


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.487377,0.866667
2,No log,0.495575,0.876190
3,No log,0.507245,0.885714


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: importance, index, hypothesis, text, Unnamed: 0. If importance, index, hypothesis, text, Unnamed: 0 are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 32


Saving model checkpoint to deberta-base-fine-tunned-neg-mnli/checkpoint-30
Configuration saved in deberta-base-fine-tunned-neg-mnli/checkpoint-30/config.json
Model weights saved in deberta-base-fine-tunned-neg-mnli/checkpoint-30/pytorch_model.bin
tokenizer config file saved in deberta-base-fine-tunned-neg-mnli/checkpoint-30/tokenizer_config.json
Special tokens file saved in deberta-base-fine-tunned-neg-mnli/checkpoint-30/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: importance, index, hypothesis, text, Unnamed: 0. If importance, index, hypothesis, text, Unnamed: 0 are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 32
Saving model checkpoint to deberta-base-fine-tunned-neg-mnli/checkpoint-60
Configuration saved in deberta-base-fine-tunned-neg-mnli

TrainOutput(global_step=90, training_loss=0.42985564337836374, metrics={'train_runtime': 68.8798, 'train_samples_per_second': 41.159, 'train_steps_per_second': 1.307, 'total_flos': 100318070248200.0, 'train_loss': 0.42985564337836374, 'epoch': 3.0})

## Evaluate with the fine-tunned model

In [ ]:
# download the checkpoint
checkpoint = "deberta-base-fine-tunned-neg-mnli/checkpoint-90"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/MNLI/dev_mismatched.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.8684906427990235

| Accuracy:  0.8621847030105777


### test on important and unimportante sentences

In [ ]:
# # reload the dataset
# data_files = './data/mnli_completed_with_importance.tsv'
# neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
# neg_datasets

In [ ]:
test_neg_datasets = neg_datasets['test']

In [ ]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 450
})

In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.93

| Accuracy:  0.8044444444444444


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.7675840978593272


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.881578947368421


# Deberta fine-tunning in snli ( original + negative sentences ) 

## preparation

In [1]:
!nvidia-smi

Fri Sep  2 12:20:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/memoire'

/content/drive/MyDrive/memoire


In [4]:
%%bash
pip install datasets
pip install transformers
pip install deberta
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=611457c22798c8e7e6e8ef9ee222ff42d509da6e3c8d776a67bc64559e6b28e8
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=f79f41f6d1ddca334183b8c1ff265930132782275741d25ea86793b33ca80df2
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built seqeva

In [5]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AdamW
import evaluate
from torch.utils.data import DataLoader

## evaluate deberta_snli

In fact, there is one ready to use

### fine-tune deberta base snli




### download datasets

In [ ]:
# download the data
datasets = load_dataset("snli")
datasets

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 550152
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
})

In [ ]:
print(set(datasets['train']['label']))

{0, 1, 2, -1}


In [ ]:
datasets = datasets.filter(lambda expression:expression['label']!=-1)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
print(set(datasets['train']['label']))

{0, 1, 2}


### prepare the model and the tokenizer

In [ ]:
# download the checkpoint
checkpoint = "microsoft/deberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

### fine-tunning on snli

In [ ]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9824
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 549367
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9842
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "deberta-base-snli", 
    evaluation_strategy="epoch",
    save_strategy = 'epoch',
    learning_rate=2e-5,
    num_train_epochs = 3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)

In [ ]:
# def compute_metrics(eval_preds):
#     metric = evaluate.load('glue', 'mnli')
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
#     return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 549367
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 206013


Epoch,Training Loss,Validation Loss
1,0.367400,0.339138
2,0.338200,0.329906


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 8
Saving model checkpoint to deberta-base-snli/checkpoint-68671
Configuration saved in deberta-base-snli/checkpoint-68671/config.json
Model weights saved in deberta-base-snli/checkpoint-68671/pytorch_model.bin
tokenizer config file saved in deberta-base-snli/checkpoint-68671/tokenizer_config.json
Special tokens file saved in deberta-base-snli/checkpoint-68671/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `DebertaForSequenceClassification.forwa

Epoch,Training Loss,Validation Loss
1,0.367400,0.339138
2,0.338200,0.329906
3,0.267000,0.379924


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 8
Saving model checkpoint to deberta-base-snli/checkpoint-206013
Configuration saved in deberta-base-snli/checkpoint-206013/config.json
Model weights saved in deberta-base-snli/checkpoint-206013/pytorch_model.bin
tokenizer config file saved in deberta-base-snli/checkpoint-206013/tokenizer_config.json
Special tokens file saved in deberta-base-snli/checkpoint-206013/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=206013, training_loss=0.3422774581987228, metrics={'train_runtime': 19224.0812, 'train_samples_per_second': 85.731, 'train_steps_per_second': 10.716, 'total_flos': 3.835099571510295e+16, 'train_loss': 0.3422774581987228, 'epoch': 3.0})

### test

In [12]:
raw_inputs = [
    "I love this.",
    "I like this.",
]
inputs = tokenizer(raw_inputs[0], raw_inputs[1], padding=True, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[  1, 100, 657,  42,   4,   2, 100, 101,  42,   4,   2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
outputs = model(**inputs.to(model.device))
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.9541, -0.9113, -4.1359]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
print(outputs.logits.shape)

torch.Size([1, 3])


In [15]:
label_map = model.config.id2label
label_map

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

In [16]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
print(label_map[predictions])

LABEL_0


### evaluate on dev

In [94]:
# download the checkpoint
checkpoint = "./deberta-base-snli/checkpoint-206013"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Didn't find file ./deberta-base-snli/checkpoint-206013/added_tokens.json. We won't load it.
loading file ./deberta-base-snli/checkpoint-206013/vocab.json
loading file ./deberta-base-snli/checkpoint-206013/merges.txt
loading file ./deberta-base-snli/checkpoint-206013/tokenizer.json
loading file None
loading file ./deberta-base-snli/checkpoint-206013/special_tokens_map.json
loading file ./deberta-base-snli/checkpoint-206013/tokenizer_config.json
loading configuration file ./deberta-base-snli/checkpoint-206013/config.json
Model config DebertaConfig {
  "_name_or_path": "./deberta-base-snli/checkpoint-206013",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2

In [17]:
# evaluate on snli_1.0_dev.tsv
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('corpus/SNLI/snli_1.0_dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[5], tokens[6], tokens[0]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# 0.334

| Accuracy:  0.9017


In [18]:
# evaluate on negation_benchmark
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('data/snli_completed_with_importance_neg_only.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[2], tokens[3], tokens[4]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# | Accuracy:  0.16533333333333333

| Accuracy:  0.5353333333333333


## Fine-tunning in a negative sentences dataset

### Download the datasets

In [96]:
data_files = './data/snli_completed_with_importance_neg_only.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 1500
})

In [97]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'index': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None)}

In [98]:
label_map = {'entailment':0 , 'neutral':1 , 'contradiction':2}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [99]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [100]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 1500
})

In [101]:
neg_datasets[1]

{'Unnamed: 0': 1,
 'index': 2,
 'text': 'A barefoot young girl in a pink gown is asleep on a hard wood floor cuddling her baby doll.',
 'hypothesis': 'Her doll does not bring her comfort.',
 'importance': '1',
 'labels': 2}

In [102]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [103]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [104]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 105
    })
})

### prepare trainer evironment

In [105]:
# download the checkpoint
# checkpoint = "./deberta-base-snli/checkpoint-206013"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [106]:
def tokenize_function(example):
    return tokenizer(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [107]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [108]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "deberta-base-fine-tunned-neg-snli", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [109]:
# def compute_metrics(eval_preds):
#     metric = evaluate.load("glue", "mnli")
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
#     return metric.compute(predictions=predictions, references=labels)

In [110]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

In [111]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, importance, Unnamed: 0, index, text. If hypothesis, importance, Unnamed: 0, index, text are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 945
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 357


Epoch,Training Loss,Validation Loss
1,No log,0.735424
2,No log,0.638529
3,No log,0.614422


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, importance, Unnamed: 0, index, text. If hypothesis, importance, Unnamed: 0, index, text are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 8
Saving model checkpoint to deberta-base-fine-tunned-neg-snli/checkpoint-119
Configuration saved in deberta-base-fine-tunned-neg-snli/checkpoint-119/config.json
Model weights saved in deberta-base-fine-tunned-neg-snli/checkpoint-119/pytorch_model.bin
tokenizer config file saved in deberta-base-fine-tunned-neg-snli/checkpoint-119/tokenizer_config.json
Special tokens file saved in deberta-base-fine-tunned-neg-snli/checkpoint-119/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` a

TrainOutput(global_step=357, training_loss=0.778044884946166, metrics={'train_runtime': 58.002, 'train_samples_per_second': 48.878, 'train_steps_per_second': 6.155, 'total_flos': 60179224884750.0, 'train_loss': 0.778044884946166, 'epoch': 3.0})

## Evaluate with the fine-tunned model

In [ ]:
# download the checkpoint
checkpoint = "deberta-base-fine-tunned-neg-snli/checkpoint-357"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [119]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/SNLI/snli_1.0_dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[5], tokens[6], tokens[0]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8769


### test on important and unimportante sentences

In [120]:
# # reload the dataset
# data_files = './data/mnli_completed_with_importance.tsv'
# neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
# neg_datasets

In [121]:
test_neg_datasets = neg_datasets['test']

In [122]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [123]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 450
})

In [124]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))


| Accuracy:  0.7466666666666667


In [125]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))



| Accuracy:  0.734375


In [126]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))


| Accuracy:  0.7578947368421053


# Deberta fine-tunning in rte ( original + negative sentences ) 

## preparation

In [ ]:
!nvidia-smi

Wed Aug 31 15:55:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/memoire'

/content/drive/MyDrive/memoire


In [ ]:
%%bash
pip install datasets
pip install transformers
pip install deberta
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=b545171c90397b548940f9681d7d558c44083fafc9eb2b214c8e20cf8f7fd200
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=3560ebd4a3c1bdc9bd31f468000f0c3a3ffe45d0492141ed3bb00c5e338e4b81
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built seqeva

In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AdamW
import evaluate
from torch.utils.data import DataLoader

## evaluate debera_rte


### download datasets

In [42]:
# download the data
datasets = load_dataset("glue", "rte")

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/rte/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [43]:
# show the data structure
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [44]:
print(set(datasets['train']['label']))

{0, 1}


### prepare the model and the tokenizer

In [45]:
# download the checkpoint
checkpoint = "microsoft/deberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

https://huggingface.co/microsoft/deberta-base/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmplt4o8utu


storing https://huggingface.co/microsoft/deberta-base/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c2bc27a1c7529c177696ff76b1e74cba8667be14e202359f20f9114e407f43e2.a39abb1c6179fb264c2db685f9a056b7cb8d4bc48d729888d292a2280debf8e2
creating metadata file for /root/.cache/huggingface/transformers/c2bc27a1c7529c177696ff76b1e74cba8667be14e202359f20f9114e407f43e2.a39abb1c6179fb264c2db685f9a056b7cb8d4bc48d729888d292a2280debf8e2
https://huggingface.co/microsoft/deberta-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpnyfxom4y


storing https://huggingface.co/microsoft/deberta-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
creating metadata file for /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
loading configuration file https://huggingface.co/microsoft/deberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e313266bff73867debdfa78c78a9a4966d5e78281ac4ed7048c178b16a37eba7.fb501413b9cef9cef6babdc543bb4153cbec58d52bce077647efba3e3f14ccf3
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps":

storing https://huggingface.co/microsoft/deberta-base/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/ce0ac094af27cf80bbf403595a6d47f1fc632981bf1d4c5bf69968568cbea410.e8ad27cc324bb0dc448d4d95f63e48f72688fb318a4c4c3f623485621b0b515c
creating metadata file for /root/.cache/huggingface/transformers/ce0ac094af27cf80bbf403595a6d47f1fc632981bf1d4c5bf69968568cbea410.e8ad27cc324bb0dc448d4d95f63e48f72688fb318a4c4c3f623485621b0b515c
https://huggingface.co/microsoft/deberta-base/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpiuub1nzz


storing https://huggingface.co/microsoft/deberta-base/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/05056f257c8d2b63ad16fd26f847c9ab9ee34e33cdfad926e132be824b237869.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/05056f257c8d2b63ad16fd26f847c9ab9ee34e33cdfad926e132be824b237869.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/microsoft/deberta-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/ce0ac094af27cf80bbf403595a6d47f1fc632981bf1d4c5bf69968568cbea410.e8ad27cc324bb0dc448d4d95f63e48f72688fb318a4c4c3f623485621b0b515c
loading file https://huggingface.co/microsoft/deberta-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/05056f257c8d2b63ad16fd26f847c9ab9ee34e33cdfad926e132be824b237869.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file http

storing https://huggingface.co/microsoft/deberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/dde0725208c11536042f6f416c538792d44a2d57d1ae399bbd1bc5867e02c465.0a3ec262cb3d4f634c72ce55f2766bb88771e6499b2512830e2e63bf19dbf97a
creating metadata file for /root/.cache/huggingface/transformers/dde0725208c11536042f6f416c538792d44a2d57d1ae399bbd1bc5867e02c465.0a3ec262cb3d4f634c72ce55f2766bb88771e6499b2512830e2e63bf19dbf97a
loading weights file https://huggingface.co/microsoft/deberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/dde0725208c11536042f6f416c538792d44a2d57d1ae399bbd1bc5867e02c465.0a3ec262cb3d4f634c72ce55f2766bb88771e6499b2512830e2e63bf19dbf97a
Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head

### fine-tunning on rte

In [46]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [47]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "deberta-base-rte", 
    evaluation_strategy="epoch",
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [48]:
def compute_metrics(eval_preds):
    metric = evaluate.load('glue', 'rte')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [49]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [50]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2. If sentence1, idx, sentence2 are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2490
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 936


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689361,0.555957
2,0.671400,0.597721,0.693141
3,0.671400,0.640855,0.707581


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2. If sentence1, idx, sentence2 are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 277
  Batch size = 8


Saving model checkpoint to deberta-base-rte/checkpoint-312
Configuration saved in deberta-base-rte/checkpoint-312/config.json
Model weights saved in deberta-base-rte/checkpoint-312/pytorch_model.bin
tokenizer config file saved in deberta-base-rte/checkpoint-312/tokenizer_config.json
Special tokens file saved in deberta-base-rte/checkpoint-312/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2. If sentence1, idx, sentence2 are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 277
  Batch size = 8
Saving model checkpoint to deberta-base-rte/checkpoint-624
Configuration saved in deberta-base-rte/checkpoint-624/config.json
Model weights saved in deberta-base-rte/checkpoint-624/pytorch_model.bin
tokenizer config file saved in deberta-base-rte/checkpoin

TrainOutput(global_step=936, training_loss=0.5998325999985393, metrics={'train_runtime': 207.4212, 'train_samples_per_second': 36.014, 'train_steps_per_second': 4.513, 'total_flos': 623951265021432.0, 'train_loss': 0.5998325999985393, 'epoch': 3.0})

### test

In [57]:
raw_inputs = [
    "I hate this.",
    "I like this so much!",
]
inputs = tokenizer(raw_inputs[0], raw_inputs[1], padding=True, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[   1,  100, 4157,   42,    4,    2,  100,  101,   42,   98,  203,  328,
            2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [58]:
outputs = model(**inputs.to(model.device))
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.6293,  0.7117]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [59]:
print(outputs.logits.shape)

torch.Size([1, 2])


In [60]:
label_map = model.config.id2label
label_map

{0: 'LABEL_0', 1: 'LABEL_1'}

In [61]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
print(label_map[predictions])

LABEL_1


### evaluate on dev

In [51]:
# download the checkpoint
checkpoint = "./deberta-base-rte/checkpoint-936"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Didn't find file ./deberta-base-rte/checkpoint-936/added_tokens.json. We won't load it.
loading file ./deberta-base-rte/checkpoint-936/vocab.json
loading file ./deberta-base-rte/checkpoint-936/merges.txt
loading file ./deberta-base-rte/checkpoint-936/tokenizer.json
loading file None
loading file ./deberta-base-rte/checkpoint-936/special_tokens_map.json
loading file ./deberta-base-rte/checkpoint-936/tokenizer_config.json
loading configuration file ./deberta-base-rte/checkpoint-936/config.json
Model config DebertaConfig {
  "_name_or_path": "./deberta-base-rte/checkpoint-936",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "p

In [62]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('corpus/RTE/dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# | Accuracy:  0.7075812274368231

| Accuracy:  0.7075812274368231


In [64]:
# evaluate on negation_benchmark
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('data/rte_completed_with_importance_neg_only.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[2], tokens[3], tokens[4]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

# | Accuracy:  0.6033333333333334


| Accuracy:  0.6033333333333334


## Fine-tunning in a negative sentences dataset

### Download the datasets

In [65]:
data_files = './data/rte_completed_with_importance_neg_only.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2da8e5812fadb932/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 1500
})

In [67]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'index': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None)}

In [66]:
label_map = {'not_entailment':1, 'entailment':0}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [68]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [69]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 1500
})

In [70]:
neg_datasets[1]

{'Unnamed: 0': 1,
 'index': 2,
 'text': "The most surprising news story of the past week must be the 'wonderful' story that the AIDS epidemic in India is under control.",
 'hypothesis': 'Polio is not under control in the world.',
 'importance': '0',
 'labels': 1}

In [71]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [72]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [73]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 105
    })
})

### prepare trainer evironment

In [ ]:
# download the checkpoint
# checkpoint = "./deberta-based-large-rte/checkpoint-936"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [74]:
def tokenize_function(example):
    return tokenizer(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [75]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 945
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [77]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "deberta-rte-fine-tunned-neg-rte", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [79]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "rte")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [80]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [81]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, importance, Unnamed: 0, index, text. If hypothesis, importance, Unnamed: 0, index, text are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 945
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1190


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.452778,0.742857
2,No log,0.538896,0.742857
3,No log,0.624990,0.733333
4,No log,0.932212,0.742857
5,0.413000,0.936956,0.752381
6,0.413000,1.273300,0.723810
7,0.413000,1.326535,0.761905
8,0.413000,1.349466,0.752381
9,0.180900,1.355267,0.761905
10,0.180900,1.379769,0.752381


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: hypothesis, importance, Unnamed: 0, index, text. If hypothesis, importance, Unnamed: 0, index, text are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 8
Saving model checkpoint to deberta-rte-fine-tunned-neg-rte/checkpoint-119
Configuration saved in deberta-rte-fine-tunned-neg-rte/checkpoint-119/config.json
Model weights saved in deberta-rte-fine-tunned-neg-rte/checkpoint-119/pytorch_model.bin
tokenizer config file saved in deberta-rte-fine-tunned-neg-rte/checkpoint-119/tokenizer_config.json
Special tokens file saved in deberta-rte-fine-tunned-neg-rte/checkpoint-119/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have be

TrainOutput(global_step=1190, training_loss=0.26598326098017333, metrics={'train_runtime': 240.7266, 'train_samples_per_second': 39.256, 'train_steps_per_second': 4.943, 'total_flos': 338623874236152.0, 'train_loss': 0.26598326098017333, 'epoch': 10.0})

## Evaluate with the fine-tunned model

In [82]:
# download the checkpoint
checkpoint = "deberta-rte-fine-tunned-neg-rte/checkpoint-1190"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Didn't find file deberta-rte-fine-tunned-neg-rte/checkpoint-1190/added_tokens.json. We won't load it.
loading file deberta-rte-fine-tunned-neg-rte/checkpoint-1190/vocab.json
loading file deberta-rte-fine-tunned-neg-rte/checkpoint-1190/merges.txt
loading file deberta-rte-fine-tunned-neg-rte/checkpoint-1190/tokenizer.json
loading file None
loading file deberta-rte-fine-tunned-neg-rte/checkpoint-1190/special_tokens_map.json
loading file deberta-rte-fine-tunned-neg-rte/checkpoint-1190/tokenizer_config.json
loading configuration file deberta-rte-fine-tunned-neg-rte/checkpoint-1190/config.json
Model config DebertaConfig {
  "_name_or_path": "deberta-rte-fine-tunned-neg-rte/checkpoint-1190",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_

In [83]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/RTE/dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.6462093862815884


### test on important and unimportante sentences

In [84]:
# # reload the dataset
# data_files = './data/mnli_completed_with_importance.tsv'
# neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
# neg_datasets

In [85]:
test_neg_datasets = neg_datasets['test']

In [86]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [87]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'index', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 450
})

In [88]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.93

| Accuracy:  0.7688888888888888


In [89]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.688034188034188


In [90]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8679245283018868
